<a href="https://colab.research.google.com/github/meehal0203/Home_sales/blob/main/Home_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [783 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,080 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:13 http://security.ubuntu.com/u

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_sales.show(5)


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [ ]:
# 2. Create a temporary view of the DataFrame.

home_sales.createOrReplaceTempView("sales_data")

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

avg_4_bed = spark.sql("""SELECT YEAR(date)AS Year, ROUND(avg(price),2)AS AvgPrice FROM sales_data WHERE bedrooms==4 GROUP BY YEAR(date)ORDER BY YEAR(date)""")
avg_4_bed.show()

+----+---------+
|Year| AvgPrice|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
avg_price_3b_3ba = spark.sql("""SELECT date_built AS Date_Built,
                                 ROUND(AVG(price),2)
                                 AS AvgPrice FROM sales_data
                                 WHERE bedrooms == 3
                                 AND bathrooms == 3
                                 GROUP BY date_built
                                 ORDER BY date_built DESC""")
avg_price_3b_3ba.show()



+----------+---------+
|Date_Built| AvgPrice|
+----------+---------+
|      2017|292676.79|
|      2016|290555.07|
|      2015| 288770.3|
|      2014|290852.27|
|      2013|295962.27|
|      2012|293683.19|
|      2011|291117.47|
|      2010|292859.62|
+----------+---------+



In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
avg_price_3b_3ba_2flr = spark.sql("""SELECT date_built AS Date_Built,
                                     ROUND(AVG(price),2)
                                     AS AvgPrice
                                     FROM sales_data
                                     WHERE bedrooms == 3
                                     AND bathrooms == 3 AND floors==2
                                     AND sqft_living >=2000
                                     GROUP BY date_built
                                     ORDER BY date_built DESC""")
avg_price_3b_3ba_2flr.show()

+----------+---------+
|Date_Built| AvgPrice|
+----------+---------+
|      2017|280317.58|
|      2016| 293965.1|
|      2015|297609.97|
|      2014|298264.72|
|      2013|303676.79|
|      2012|307539.97|
|      2011|276553.81|
|      2010|285010.22|
+----------+---------+



In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""SELECT view AS ViewRating, ROUND(AVG(price), 2)
              AS AvgPrice
              FROM sales_data
              GROUP BY view
              HAVING AVG(price) >= 350000
              ORDER BY view DESC""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----------+----------+
|ViewRating|  AvgPrice|
+----------+----------+
|        99|1061201.42|
|        98|1053739.33|
|        97|1129040.15|
|        96|1017815.92|
|        95| 1054325.6|
|        94| 1033536.2|
|        93|1026006.06|
|        92| 970402.55|
|        91|1137372.73|
|        90|1062654.16|
|        89|1107839.15|
|        88|1031719.35|
|        87| 1072285.2|
|        86|1070444.25|
|        85|1056336.74|
|        84|1117233.13|
|        83|1033965.93|
|        82| 1063498.0|
|        81|1053472.79|
|        80| 991767.38|
+----------+----------+
only showing top 20 rows

--- 1.1479198932647705 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table sales_data")


DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('sales_data')

True

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.

start_time = time.time()

spark.sql("""SELECT view AS ViewRating, ROUND(AVG(price), 2)
              AS AvgPrice
              FROM sales_data
              GROUP BY view
              HAVING AVG(price) >= 350000
              ORDER BY view DESC""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----------+----------+
|ViewRating|  AvgPrice|
+----------+----------+
|        99|1061201.42|
|        98|1053739.33|
|        97|1129040.15|
|        96|1017815.92|
|        95| 1054325.6|
|        94| 1033536.2|
|        93|1026006.06|
|        92| 970402.55|
|        91|1137372.73|
|        90|1062654.16|
|        89|1107839.15|
|        88|1031719.35|
|        87| 1072285.2|
|        86|1070444.25|
|        85|1056336.74|
|        84|1117233.13|
|        83|1033965.93|
|        82| 1063498.0|
|        81|1053472.79|
|        80| 991767.38|
+----------+----------+
only showing top 20 rows

--- 0.6008601188659668 seconds ---


In [ ]:
# Determine the runtime and compare it to the uncached runtime.
print("the cached runtime 0.6 seconds was almost 50% faster than the original runtime of 1.14 seconds")


the cached runtime 0.6 seconds was almost 50% faster than the original runtime of 1.14 seconds


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales.write.parquet('sales_parquet', mode='overwrite')


In [ ]:
# 11. Read the parquet formatted data.

p_home_sales = spark.read.parquet('sales_parquet')

In [ ]:
# 12. Create a temporary table for the parquet data.
p_home_sales.createOrReplaceTempView("p_home_sales_p")

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.

start_time = time.time()

spark.sql("""SELECT view AS ViewRating, ROUND(AVG(price), 2)
              AS AvgPrice
              FROM p_home_sales_p
              GROUP BY view
              HAVING AVG(price) >= 350000
              ORDER BY view DESC""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----------+----------+
|ViewRating|  AvgPrice|
+----------+----------+
|        99|1061201.42|
|        98|1053739.33|
|        97|1129040.15|
|        96|1017815.92|
|        95| 1054325.6|
|        94| 1033536.2|
|        93|1026006.06|
|        92| 970402.55|
|        91|1137372.73|
|        90|1062654.16|
|        89|1107839.15|
|        88|1031719.35|
|        87| 1072285.2|
|        86|1070444.25|
|        85|1056336.74|
|        84|1117233.13|
|        83|1033965.93|
|        82| 1063498.0|
|        81|1053472.79|
|        80| 991767.38|
+----------+----------+
only showing top 20 rows

--- 0.59564208984375 seconds ---


In [44]:
# Determine the runtime and compare it to the cached runtime.
print("the parquet runtime 0.59 seconds is almost identical to the cached runtime of 0.60 seconds")


the parquet runtime 0.59 seconds is almost identical to the cached runtime of 0.60 seconds


In [45]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table sales")


DataFrame[]

In [46]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('sales')


False